# guppy and hugr to qir conversion and submission to H1 and H2


This example shows how to convert guppy to qir which can be submitted directly to H1 and H2 device, emulator and syntax checker.

You need to install hugr-qir, guppy and pytket-quantinuum for this notebook to work.


In [2]:
# You can write you guppy directly in a notebook or in a separate file
import sys
from typing import no_type_check

from guppylang import guppy, quantum, qubit
from guppylang.std.angles import angles
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure

guppy.load(quantum)
guppy.load(angles)


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    h(q0)
    h(q1)

    b0 = measure(q0)
    b1 = measure(q1)
    b2 = b0 ^ b1

    result("0", b2)





In [3]:
# compile your guppy to hugr
hugr = guppy.get_module().compile().package.to_bytes()


In [4]:
# convert hugr to qir
# the parameter validate_qir=True automatically checks the generated QIR to capture most of the issues that could happen.
# this will show an error message with more details about the problem

from hugr_qir.hugr_to_qir import hugr_to_qir

guppy_qir = hugr_to_qir(hugr, validate_qir=True)


In [5]:
# the QIR is returned as a string
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1

define void @__hugr__.main.1() #0 {
alloca_block:
  call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* null)
  call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* null)
  call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  %0 = call i1 @__quantum__qis__read_result__body(%Result* null)
  call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__mz__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Result* inttoptr (i64 1 to %Result*))
  %1 = call i1 @__quantum__qis__read_result__body(%Result* inttoptr (i64 1 to %Result*))
  %2 = xor i1 %0, %1
  call v

In [6]:
# set up a pytket-quantinnum backend
# This required credentials for the device

from pytket.extensions.quantinuum import QuantinuumBackend, Language


backend = QuantinuumBackend(device_name="H1-1SC")
backend.login()


Enter your Quantinuum email:  melf.johannsen@quantinuum.com
Enter your Quantinuum password:  ········


In [8]:
# convert the qir string to base64 binary, which is expected by the device

from llvmlite.binding import create_context, parse_assembly
from base64 import b64encode

ctx = create_context()
module = parse_assembly(guppy_qir, context=ctx)
ir = module.as_bitcode()
qir = b64encode(ir).decode("utf-8")

h = backend.submit_program(Language.QIR, qir, n_shots=10)
r = backend.get_result(h)
shots = r.get_shots()
assert len(shots) == 10
print(h)
print(shots)

('5091ebd67d104e39a728c8747fdbd238', 'null', -1, 'null')
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [9]:
import sys
from typing import no_type_check

from guppylang import guppy, quantum, qubit
from guppylang.std.angles import angles
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure

guppy.load(quantum)
guppy.load(angles)


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    for _ in range(10):
        q3 = qubit()
        h(q3)
        b = measure(q3)
        if b:
            h(q0)

    result("0", measure(q0))
    result("1", measure(q1))



In [13]:
# currently it is not supported to convert guppy containing loops into QIR that is understood by H1 and H2.
# Running this example shows the expected error.

hugr = guppy.get_module().compile().package.to_bytes()

guppy_qir = hugr_to_qir(hugr, validate_qir=True)
